In [1]:
import pandas as pd
import cobra

from cobra.io import load_model

import numpy as np
import pickle

import os.path

Ponovno preberemo model Recon3D:

In [2]:
model = cobra.io.read_sbml_model(os.path.join('models','Recon3D.xml'))

Pripravimo si slovarja `id gena` --> `ime gena` in obratno:

In [3]:
gene_id_name = {} # seznam ID genov
gene_name_id = {} # seznam imen genov

for g in model.genes[1:]:
    gene_id_name[g.id] = g.name
    gene_name_id[g.name] = g.id

In [4]:
list(gene_id_name.items())[:5] # prvih 5 elementov v slovarju, ki ID gena poveže z imenom

[('26_AT1', 'AOC1'),
 ('314_AT1', 'AOC2'),
 ('8639_AT1', 'AOC3'),
 ('314_AT2', 'AOC2'),
 ('1591_AT1', 'CYP24A1')]

In [5]:
list(gene_name_id.items())[:5] # prvih 5 elementov v slovarju, ki ime gena poveže z ID

[('AOC1', '26_AT1'),
 ('AOC2', '314_AT2'),
 ('AOC3', '8639_AT1'),
 ('CYP24A1', '1591_AT1'),
 ('CYP27B1', '1594_AT1')]

Preberimo podatke, ki smo jih prej shranili:

In [6]:
df_control = pd.read_csv(os.path.join('data','data_control.txt'))
df_kd = pd.read_csv(os.path.join('data','data_kd.txt'))

Imenom genov pripišimo še ustrezne ID-je:

In [7]:
df_control['gene_id'] = df_control['gene'].map(lambda x: gene_name_id[x] if x in gene_name_id else "")
df_kd['gene_id'] = df_kd['gene'].map(lambda x: gene_name_id[x] if x in gene_name_id else "")

In [8]:
df_control

,gene,value,gene_id
0,OR4F5,6.13,
1,SAMD11,6.97,
2,KLHL17,7.10,
3,PLEKHN1,5.51,
4,ISG15,6.26,
...,...,...,...
21443,PCYT1A,12.46,5130_AT1
21444,CFHR2,6.13,
21445,DUX5,5.13,
21446,DUX3,4.39,


Ohranimo samo tiste vnose, ki jih imamo v modelu:

In [9]:
df_control = df_control[df_control['gene_id'] != ""]
df_kd = df_kd[df_kd['gene_id'] != ""]

In [10]:
len(df_control), len(df_kd)

(2019, 2019)

In [11]:
df_control.describe()

,value
count,2019.000000
mean,8.288940
std,3.151708
min,3.310000
25%,5.595000
50%,7.610000
75%,10.795000
max,17.540000


In [12]:
df_kd.describe()

,value
count,2019.000000
mean,8.324889
std,3.169539
min,3.230000
25%,5.605000
50%,7.680000
75%,10.890000
max,17.480000


### Integracija transkriptomskih podatkov v model

Uporabili bomo algoritem CORDA:

* [https://resendislab.github.io/corda/](https://resendislab.github.io/corda/)
* [https://github.com/resendislab/corda](https://github.com/resendislab/corda)

Funkcija za rekonstrukcijo kot argument sprejme model in preslikavo `confidence`, ki za vsako reakcijo poda mero zaupanja, da je vsebovana v modelu:
```
Allowed confidence values are -1 (absent/do not include), 0 (unknown), 1 (low confidence), 2 (medium confidence) and 3 (high confidence).
```

`confidence` si lahko pripravimo z uporabo CORDA funkcije `reaction_confidence`, ki preslika stopnje izražanja genov (po enaki lestvici kot zgoraj) v stopnje aktivnosti reakcij z uporabo pravil GPR.


Dodatno lahko nastavimo zmožnost modela, da producira biomaso:
```
met_prod (Optional[list]): Additional metabolic targets that have to be
            achieved by the model. Can be a single object or list of objects.
            List elements can be given in various forms:
            (1) A string naming a metabolite in the model will ensure that the
            metabolite can be produced.
            For instance: met_prod = ["pi_c", "atp_c"]
            (2) A dictionary of metabolite -> int will define an irreversible
            reaction that must be able to carry flux.
            For instance: met_prod = {"adp_c": -1, "pi_c": -1, "atp_c": 1}
            (3) A string representation of a reversible or irreversible
            reaction that must be able to carry flux.
            For instance: met_prod = "adp_c + pi_c -> atp_c"

```
Primer priprave niza `met_prod`:
```Python
met_prod = model.reactions.BIOMASS_maintenance.build_reaction_string()
met_prod = met_prod.replace("-->", "->")
```


In [13]:
biomass = model.reactions.BIOMASS_maintenance.build_reaction_string().replace("-->", "->")
biomass

'0.50563 ala__L_c + 0.35926 arg__L_c + 0.27942 asn__L_c + 0.35261 asp__L_c + 20.7045 atp_c + 0.020401 chsterol_c + 0.011658 clpn_hs_c + 0.039036 ctp_c + 0.046571 cys__L_c + 0.27519 g6p_c + 0.326 gln__L_c + 0.38587 glu__L_c + 0.53889 gly_c + 0.036117 gtp_c + 20.6508 h2o_c + 0.12641 his__L_c + 0.28608 ile__L_c + 0.54554 leu__L_c + 0.59211 lys__L_c + 0.15302 met__L_c + 0.023315 pail_hs_c + 0.15446 pchol_hs_c + 0.055374 pe_hs_c + 0.002914 pglyc_hs_c + 0.25947 phe__L_c + 0.41248 pro__L_c + 0.005829 ps_hs_c + 0.39253 ser__L_c + 0.017486 sphmyln_hs_c + 0.31269 thr__L_c + 0.013306 trp__L_c + 0.15967 tyr__L_c + 0.053446 utp_c + 0.35261 val__L_c -> 20.6508 adp_c + 20.6508 h_c + 20.6508 pi_c'

### Preslikava izražanja genov v prisotnosti reakcij

In [14]:
from corda import reaction_confidence
from corda import CORDA

Pomagali si bomo s funkcijo, ki bo preslikala izražanje genov v vrednosti -1, 0, 1, 2 in 3 preko vnaprej definiranih pragov.

In [15]:
def val_to_score(val, thr0, thr1, thr2, thr3):
    if val < thr0:
        return -1
    elif val < thr1:
        return 0
    elif val < thr2:
        return 1
    elif val < thr3:
        return 2
    else:
        return 3

#### Parametri rekonstrukcije
Pragove bomo definirali preko odstotne aktivnosti genov v vzrocu. Obstaja veliko različnih strategij, kako se tega problema lotiti. Nekaj več o tem si lahko preberete v [članku](https://doi.org/10.1371/journal.pcbi.1007185).

In [16]:
require_biomass = True
perc0 = 25 # below this ... inactive
perc1 = 50 # else, below this ... not sure
perc2 = 75 # else, below this ...  low confidence
perc3 = 95 # else, below this ... medium confidence
# else high confidence (above perc3)

In [18]:
folder = "models"
folder

'models'

### Kontrola
Za vsako skupino bomo določili lastne pragove glede na aktivnosti genov:

In [19]:
thr0 = np.percentile(df_control.value, perc0)
thr1 = np.percentile(df_control.value, perc1)
thr2 = np.percentile(df_control.value, perc2)
thr3 = np.percentile(df_control.value, perc3)

Najprej preslikajmo intenziteto izražanja genov v skupine:

In [20]:
#gene_conf_control = {g_id:-1 if val < thr0 else 1 if val < thr1 else 2 if val < thr2 else 3 for (g_id, val) in zip(df_control['gene_id'], df_control['value'])}
gene_conf_control = {g_id : val_to_score(val, thr0, thr1, thr2, thr3) for (g_id, val) in zip(df_control['gene_id'], df_control['value'])}

Potem pa skupine genov uporabimo pri razvrščanju reakcij v skupine z uporabo CORDA funkcije `reaction_confidence`, ki pri preslikavi upošteva pravila GPR.

In [21]:
conf_control = {}
for r in model.reactions:
    conf_control[r.id] = reaction_confidence(r, gene_conf_control)

Potem poženimo integracijo z algoritmom:

**POZOR: časovno potratno! Spodnje celice lahko preskočiš in podatke prebereš iz datoteke (spodaj).**

In [ ]:
if require_biomass:
    opt_control = CORDA(model, conf_control, met_prod = biomass)
else:
    opt_control = CORDA(model, conf_control)
opt_control.build()
print(opt_control)

Model pretvorimo v COBRA zapis:

In [ ]:
model_control = opt_control.cobra_model()

In mu dodajmo še biomaso:

In [ ]:
if require_biomass:
    model_control.add_reactions([model.reactions.BIOMASS_maintenance])
    model_control.objective=model.objective

Model lahko zdaj shranimo v SBML zapis:

In [ ]:
cobra.io.write_sbml_model(model_control, os.path.join(f'{folder}','model_control.xml'))

**Branje iz datoteke (če ste prejšnje korake preskočili).**

In [27]:
model_control = cobra.io.read_sbml_model(os.path.join(f'{folder}','model_control.xml'))

In [28]:
model_control.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
2pg_e,EX_2pg_e,992.7,3,3.90%
HC00250_e,EX_HC00250_e,992.7,0,0.00%
asn__L_e,EX_asn__L_e,27.01,4,0.14%
chsterol_e,EX_chsterol_e,1.972,27,0.07%
crm_hs_e,EX_crm_hs_e,1.69,19,0.04%
fol_e,EX_fol_e,258.2,19,6.43%
ile__L_e,EX_ile__L_e,27.65,6,0.22%
imp_e,EX_imp_e,3.491,10,0.05%
inost_e,EX_inost_e,2.254,6,0.02%
leu__L_e,EX_leu__L_e,52.74,6,0.41%


### Knockdown

Ponovimo podoben postopek za podatke z utišanim onkogenom:

In [34]:
thr0 = np.percentile(df_kd.value, perc0)
thr1 = np.percentile(df_kd.value, perc1)
thr2 = np.percentile(df_kd.value, perc2)
thr3 = np.percentile(df_kd.value, perc3)

In [ ]:
gene_conf_kd = {g_id : val_to_score(val, thr0, thr1, thr2, thr3) for (g_id, val) in zip(df_kd['gene_id'], df_kd['value'])}

In [ ]:
conf_kd = {}
for r in model.reactions:
    conf_kd[r.id] = reaction_confidence(r, gene_conf_kd)

**POZOR: časovno potratno! Spodnje celice lahko preskočiš in podatke prebereš iz datoteke (spodaj).**

In [ ]:
if require_biomass:
    opt_kd = CORDA(model, conf_kd, met_prod = biomass)
else:
    opt_kd = CORDA(model, conf_kd)
opt_kd.build()
print(opt_kd)

In [ ]:
model_kd = opt_kd.cobra_model()

In [ ]:
if require_biomass:
    model_kd.add_reactions([model.reactions.BIOMASS_maintenance])
    model_kd.objective=model.objective   

In [ ]:
cobra.io.write_sbml_model(model_kd, os.path.join(f'{folder}','model_kd.xml'))

**Branje iz datoteke (če ste prejšnje korake preskočili).**

In [22]:
model_kd = cobra.io.read_sbml_model(os.path.join(f'{folder}','model_kd.xml'))

In [23]:
model_kd.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
2pg_e,EX_2pg_e,1000,3,4.24%
5aop_e,EX_5aop_e,171.7,5,1.21%
HC00250_e,EX_HC00250_e,990.4,0,0.00%
bilirub_e,EX_bilirub_e,11.84,33,0.55%
chsterol_e,EX_chsterol_e,2.611,27,0.10%
crm_hs_e,EX_crm_hs_e,2.238,19,0.06%
dtmp_e,EX_dtmp_e,500,10,7.06%
fol_e,EX_fol_e,88.02,19,2.36%
glygn2_e,EX_glygn2_e,4.402,66,0.41%
h2o2_e,EX_h2o2_e,492.2,0,0.00%
